In [ ]:
import hyperspy.api as hs
import snmfem.data_analysis as da
import snmfem.experiments as exp

In [ ]:
spim = hs.load("71GPa_subsolidus.hspy")
da.set_analysis_parameters(
    spim,
    density= 4.5,
    thickness= 200e-7
)

In [ ]:
haadf_ROI=hs.roi.RectangularROI(10*spim.axes_manager[0].scale,
                            10*spim.axes_manager[1].scale,
                            40*spim.axes_manager[0].scale,
                            40*spim.axes_manager[1].scale)
# arguments = left, top, right, bottom in pixels*scale
spim.plot()
spim_ROI=haadf_ROI.interactive(spim)
# The HAADF and SPIM windows with their respective ROIs. The two ROIs are connected so that the selected area in the HAADF is the same as the one on the SPIM.

spectrum=hs.interactive(spim_ROI.mean,
            event=spim_ROI.axes_manager.events.any_axis_changed,
            recompute_out_event=None)
spectrum.plot(True)

In [ ]:
left,right,top,bottom = da.get_ROI_ranges(spim,haadf_ROI)
ind_list = da.build_index_list(left,right,top,bottom,spim.axes_manager[0].size)

In [ ]:
Xflat, shape_2d, mod_pars, g_pars = da.load_hs_data(spim)
exp_params = ({"name": "snmfem smooth 3", "method": "SmoothNMF", "k" : 3},{"lambda_L" : 3.0,"max_iter" : 20})
exp = da.build_analysis(exp_params[0],exp_params[1])
G = da.build_model(mod_pars, g_pars, G_func = True)
G, P, A, losses = da.run_analysis(Xflat, G, exp, shape_2d = shape_2d,mod_pars = mod_pars,g_pars = g_pars, ind_list = ind_list)


In [ ]:
with open("test.txt","w") as f : 
    for elt in ind_list : 
        f.write(str(elt))
        f.write(" ")